In [1]:

import numpy as np
import pickle as pickle

In [2]:
deapSubList=['01','02','03','04','05','06','07','08','09','10','11','12','13','14',\
         '15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']
#用一个字典保存 通道下标对应 9 * 9 矩阵的下标
channel_to_1020={0:[0,3],1:[1,3],2:[2,2],3:[2,0],4:[3,1],5:[3,3],6:[4,2],7:[4,0],8:[5,1],9:[5,3],10:[6,3],11:[6,0],12:[7,3],
                13:[8,3],14:[7,4],15:[6,4],16:[0,5],17:[1,5],18:[2,4],19:[2,6],20:[2,8],21:[3,7],22:[3,5],23:[4,4],24:[4,6],
                25:[4,8],26:[5,7],27:[5,5],28:[6,6],29:[6,8],30:[7,5],31:[8,5]}

# 把EEG信号映射到10-20系统中，并把其他信号提取出来备用
# EEG 和 其他特征信号均进行了去基线的操作，以后进行训练的时候更方便
# 并把EEG特征标准化在放入，其他特征先不标准化

In [3]:
# 把基线按照1s窗口分割，每秒一次分割，也就是128次一分割
def base_split(baseline_data):
    deap_baseline_split = np.zeros((40,3,40,128))
    for i in range(40):
        windows = 128//1
        part = 3
        baseli = np.stack(np.split(baseline_data[i],part,axis=1))
        deap_baseline_split[i] = baseli
    return deap_baseline_split

In [4]:
# 把特征按照基线做同样的处理，为了方便去基线
def feature_split(feature_data):
    features_split = np.zeros((40,60,40,128))
    for i in range(40):
        windows = 128//1
        part = 60
        fea = np.stack(np.split(feature_data[i],part,axis=1))
        features_split[i] = fea
    return features_split

In [5]:
# 把（40,60,40,128）再变回去（40,40,7680），方便映射到10-20系统中的9*9矩阵中
def feature_to_oriShape(features_split_moveBasli):
    feature_move_baseline = np.zeros((40,40,7680))
    for i in range(40):
        gap = 0
        for j in range(60):
            feature_move_baseline [i][:,gap:gap+128] = features_split_moveBasli[i][j]
            gap+=128
    return feature_move_baseline

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
def EEG_to_1020():
    for subs in deapSubList:
        with open(r'D:\JupyterNotebookWorkSpace\EEG emotion recognition\dataSet\data_preprocessed_python\s'+ subs + '.dat' , 'rb') as file:

            
            allData = sub = pickle.load(file , encoding='latin1')
            allData = allData['data']
            feature_data = allData[:,:,384:]  #取所有信号的特征
            baseline_data = allData[:,:,:384] #取所有信号的基线
            deap_baseline_split = base_split(baseline_data)  #基线按照1s分割
            features_split=feature_split(feature_data)  #特征按照1s分割
            # 去掉所有信号的基线再进行保存
            temp=np.mean(deap_baseline_split,axis=1)[:,np.newaxis,:,:]
            features_split_moveBasli = features_split  - temp
            
            #把去掉基线的所有特征变回原来的形状（40,40,7680）
            all_features=feature_to_oriShape(features_split_moveBasli)
            
            eeg_data = all_features[:,:32,:]  # 把eeg特征取出，之后映射
            other_features = all_features[:,32:,:]
            # EEG 数据的形状为 （40,32,7680）
            std = StandardScaler()
            
            #这里我是先进行的标准化，之后再进行的映射到二维帧
            # ------------------------------------------------------------------------- 标准化问题
            for i in range(40):
                eeg_data[i] = std.fit_transform(eeg_data[i])
            #下面的for循环把eeg映射到10-20系统的9*9矩阵中
            # 原来是（40,32,7680）
            eeg_features =np.zeros((40,7680,9,9))
            for i in range(40):
                for j in range(32):
                    index=0
                    for k in range(7680):
                        row = channel_to_1020[j][0]
                        column = channel_to_1020[j][1]
                        #把EEG信号进行标准化
                        eeg_features[i,index,row,column]=eeg_data[i,j,k]
                        index+=1
        labels = sub["labels"][:,:2]
        out_data_dict={'eeg_features':eeg_features,
                       'other_features':other_features,'labels':labels}
        np.save('eeg_to_1020/1020sub'+subs+'.npy',out_data_dict)    

                    
EEG_to_1020()
    

In [ ]:
with open(r'D:\DEAP data\data_preprocessed_python\s'+ '01' + '.dat' , 'rb') as file:
    allData = sub = pickle.load(file , encoding='latin1')
    allData = allData['data']
    feature_data = allData[:,:,:7680]
    baseline_data = allData[:,:,7680:]
    deap_baseline_split = base_split(baseline_data)
    features_split=feature_split(feature_data)

In [18]:
#所有 特征 和 基线 的原始数据
feature_data.shape,baseline_data.shape

((40, 40, 7680), (40, 40, 384))

In [49]:
#所有 特征 和 基线 进行1s分割的数据形状
deap_baseline_split.shape,features_split.shape

((40, 3, 40, 128), (40, 60, 40, 128))

In [52]:
# 下面三个代码块用来去基线
temp=np.mean(deap_baseline_split,axis=1)[:,np.newaxis,:,:]

In [53]:
features_split_moveBasli = features_split  - temp

In [54]:
features_split_moveBasli.shape

(40, 60, 40, 128)

In [63]:
all_features=feature_to_oriShape(features_split_moveBasli)

In [64]:
all_features.shape

(40, 40, 7680)